In [2]:
# We always start with a dataset to train on. Let's download the tiny shakespeare dataset
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2023-05-07 15:12:59--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 2606:50c0:8002::154, 2606:50c0:8000::154, 2606:50c0:8001::154, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|2606:50c0:8002::154|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: 'input.txt.1'

     0K .......... .......... .......... .......... ..........  4% 2.52M 0s
    50K .......... .......... .......... .......... ..........  9% 11.6M 0s
   100K .......... .......... .......... .......... .......... 13% 5.71M 0s
   150K .......... .......... .......... .......... .......... 18% 17.0M 0s
   200K .......... .......... .......... .......... .......... 22% 33.7M 0s
   250K .......... .......... .......... .......... .......... 27% 5.17M 0s
   300K .......... .......... .......... .......... .......... 32% 37.1M 0s
   

In [3]:
# read it in to inspect it
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [3]:
print("length of dataset in characters: ", len(text))

length of dataset in characters:  1115394


In [4]:
# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [5]:
stoi = { ch:i for i,ch in enumerate(chars) } #lookup table for encoder
itos = { i:ch for i,ch in enumerate(chars) } #lookup table for decoder
encode = lambda s: [stoi[c] for c in s] #encoder
decode = lambda l: ''.join([itos[i] for i in l]) #decoder

In [6]:
print(encode("hi!"))
print(decode(encode("hi!")))

[46, 47, 2]
hi!


In [7]:
import torch
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:2])

torch.Size([1115394]) torch.int64
tensor([18, 47])


In [8]:
#Splitting into train and validation
n = int(0.9*len(data))

#90% for training
train_data = data[:n]
val_data = data[n:]

In [10]:
block_size = 8 #size of training data chunks
train_data[:block_size+1] #has multiple examples packed into it

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [11]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"when input is {context} the target: {target}")

# We want transformer to see anything from context size from 1 to block size. After block size is reached, we start chunking

when input is tensor([18]) the target: 47
when input is tensor([18, 47]) the target: 56
when input is tensor([18, 47, 56]) the target: 57
when input is tensor([18, 47, 56, 57]) the target: 58
when input is tensor([18, 47, 56, 57, 58]) the target: 1
when input is tensor([18, 47, 56, 57, 58,  1]) the target: 15
when input is tensor([18, 47, 56, 57, 58,  1, 15]) the target: 47
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target: 58


In [12]:
#Mini batches of multiple text and want to process them @ same time
torch.manual_seed(1337)
batch_size = 4 #independent sequences will we process in parallel
block_size = 8 #maximum context length

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,)) #generates 4 random indices 
    x = torch.stack([data[i:i+block_size] for i in ix]) #gets a list of 8 indices for block size and stacks them
    y = torch.stack([data[i+1:i+block_size+1] for i in ix]) #same as x but an index forward since this is the target
    return x, y

xb, yb = get_batch('train')
print('inputs: ')
print(xb.shape)
print(xb)
print('targets: ')
print(yb.shape)
print(yb)
print('----------------')

for b in range(batch_size):
    for t in range(block_size):
        context = xb[b, :t+1]
        target = yb[b,t]
        print(f"when input is {context.tolist()} the target: {target}")


inputs: 
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
targets: 
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
----------------
when input is [24] the target: 43
when input is [24, 43] the target: 58
when input is [24, 43, 58] the target: 5
when input is [24, 43, 58, 5] the target: 57
when input is [24, 43, 58, 5, 57] the target: 1
when input is [24, 43, 58, 5, 57, 1] the target: 46
when input is [24, 43, 58, 5, 57, 1, 46] the target: 43
when input is [24, 43, 58, 5, 57, 1, 46, 43] the target: 39
when input is [44] the target: 53
when input is [44, 53] the target: 56
when input is [44, 53, 56] the target: 1
when input is [44, 53, 56, 1] the target: 58
when input is [44, 53, 56, 1, 58] the target: 46
when in

In [17]:
#Bigram Language Model
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        # 
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
    
    def forward(self, idx, targets):
        #when we pass idx, every single int in input refers to embedding table
        #and find the idx'th row in the tensor. So 24 -> pluck out 24th row

        logits = self.token_embedding_table(idx) #(Batch, Time, Channel) => (4,8,65)
        B, T, C = logits.shape
        logits = logits.view(B*T, C)
        
        targets = targets.view(B*T)
        loss = F.cross_entropy(logits, targets)
        return logits, loss

In [18]:
m = BigramLanguageModel(vocab_size=vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss) #expected loss should be 4.17...

torch.Size([32, 65])
tensor(4.8786, grad_fn=<NllLossBackward0>)
